Será treinado um modelo bem simples para fazer comparação entre dois números e dizer se um é maior que outro.

Esse modelo será convertido e rodará no ESP32.

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras import Input
from tensorflow.data import Dataset
import numpy as np
import itertools

## Criando um dataset


In [ ]:
def data_generator():
    while(True):
        number1 = np.random.uniform();
        number2 = np.random.uniform();
        X = [number1, number2]
        Y = 1 if number2 > number1 else 0
        yield X, [Y]

train_dataset = tf.data.Dataset.from_generator(
    data_generator,
    output_types = (tf.float32, tf.int32),
    output_shapes=((2), (1))
)
train_dataset = train_dataset.batch(batch_size=30)

## O modelo é simples!

Não é necessário um modelo super complicado para esse problema, então vamos definir uma pequena rede neural com uma input layer e uma output layer.

É importante que a função de ativação do output seja sigmoid. Essa função de ativação trará um output de um valor entre 0 e 1.

In [ ]:
model = Sequential([
    Input(shape=(2)),
    Dense(5, activation='relu'),
    Dense(1, activation='sigmoid')
])

## Compilando o modelo
Para a função de perda será usado BinaryCrossentropy.


Crossentropy quantifica a diferença entre a distribuição de duas probabilidades.

We have a binary distribution (True or False) so we use binary crossentropy to compare the output from our model with the true distribution.

Temos uma distribuição binária (Verdadeiro ou Falso) então será usado binary crossentropy para comparar o output do modelo com a distribuição verdadeira.


In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy'])

model.summary()

Model: "sequential_20"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_34 (Dense)             (None, 5)                 15        
_________________________________________________________________
dense_35 (Dense)             (None, 1)                 6         
Total params: 21
Trainable params: 21
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(
    train_dataset,
    steps_per_epoch=1000,
    epochs=4
)

Epoch 1/4
1000/1000 [==============================] - 7s 7ms/step - loss: 0.3755 - accuracy: 0.9768
Epoch 2/4
1000/1000 [==============================] - 6s 6ms/step - loss: 0.1731 - accuracy: 0.9945
Epoch 3/4
1000/1000 [==============================] - 7s 7ms/step - loss: 0.1178 - accuracy: 0.9960
Epoch 4/4
1000/1000 [==============================] - 6s 6ms/step - loss: 0.0903 - accuracy: 0.9954


## Testando o modelo


In [ ]:
test_X = np.array([
    [0.1, 0.2],
    [0.3, 0.4],
    [0.5, 0.1],
    [0.7, 0.2]
])
Y = model.predict_on_batch(test_X)
np.set_printoptions(formatter={'float': lambda x: "{0:0.2f}".format(x)})
print(Y)

[[0.87]
 [0.87]
 [0.00]
 [0.00]]


## Exportando para TFLITE


In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
def representative_dataset_gen():
    for _ in range(10000):
        yield [
            np.array(
                [np.random.uniform(), np.random.uniform()]
            , dtype=np.float32)
        ]
converter.representative_dataset = representative_dataset_gen
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
tflite_quant_model = converter.convert()
open("converted_model.tflite", "wb").write(tflite_quant_model)

INFO:tensorflow:Assets written to: /var/folders/3_/9hvjft757dz9zxvjby_2v9j80000gn/T/tmpojx2qz2e/assets


INFO:tensorflow:Assets written to: /var/folders/3_/9hvjft757dz9zxvjby_2v9j80000gn/T/tmpojx2qz2e/assets


2176

## Converter para C++
```
xxd -i converted_model.tflite > model_data.cc
```